In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = trOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

def generate_club_name(club_type):
    llm = OpenAI(temperature = 0.7)
    
    prompt_template_name = PromptTemplate(
        input_variables=['club_type'],
        template = "Suggest five cool names for a new {club_type} club. "
    )
    name_chain = LLMChain(llm=llm, prompt = prompt_template_name)
    
    response = name_chain({'club_type' : club_type})
    return response

if __name__ == "__main__":
    print(generate_club_name("Computer Engineering"))
    

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import random

def generate_advice(situation):
    llm = OpenAI(temperature=0.7)

    # Load the Art of War text
    with open("art_of_war.txt", "r", encoding="utf-8") as file:
        art_of_war_text = file.read()

    # Randomly select a passage from the text to base the advice on
    passages = art_of_war_text.split("\n\n")
    selected_passage = random.choice(passages)

    prompt_template_advice = PromptTemplate(
        input_variables=['situation', 'selected_passage'],
        template="Given the situation: '{situation}', what advice from this passage of Sun Tzu's Art of War would be most applicable? Passage: '{selected_passage}'. Advice: "
    )
    advice_chain = LLMChain(llm=llm, prompt=prompt_template_advice, output_key="advice")

    response = advice_chain({'situation': situation, 'selected_passage': selected_passage})
    
    # Include the selected passage in the response for citation
    response['selected_passage'] = selected_passage
    return response


In [2]:
if __name__ == "__main__":
    situation = "Negotiating a business deal"
    response = generate_advice(situation)

    # Extracting advice and the cited passage
    advice = response.get('advice', 'No advice generated.')
    cited_passage = response.get('selected_passage', 'No passage selected.')

    # Formatting the output
    print(f"Situation: {situation}\n")
    print(f"Advice: {advice}\n")
    print(f"Cited from 'The Art of War':\n{cited_passage}")

Situation: Negotiating a business deal

Advice: 
The most applicable advice from this passage would be to prioritize gaining foreknowledge in the negotiation process. This means doing thorough research, understanding the other party's motivations and objectives, and anticipating potential obstacles or challenges. By having foreknowledge, one can strategize and make informed decisions that can lead to a successful negotiation and achieving objectives beyond what is typically achievable.

Cited from 'The Art of War':
4. Thus, what enables the wise sovereign and the good general to strike
and conquer, and achieve things beyond the reach of ordinary men,
is foreknowledge. 
